# Scenario 01: Simple Agent with MCP Tools

**Estimated Time**: 30 minutes

## Learning Objectives
- Create a base agent using the Microsoft Agent Framework
- Integrate MCP tools for extended capabilities
- Implement OpenTelemetry observability
- Understand the agent execution lifecycle

## Prerequisites
- Completed Scenario 00 (Environment Setup)
- Azure OpenAI API access configured

## Part 1: Environment Setup

In [1]:
# Load environment and configure paths
import sys
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")

✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop


In [2]:
# Initialize telemetry for observability
from src.common.telemetry import setup_telemetry, get_tracer
from src.common.config import get_settings

# Clear any cached settings and setup telemetry
get_settings.cache_clear()
setup_telemetry()

settings = get_settings()
print(f"✅ Telemetry initialized")
print(f"   Azure OpenAI: {'Configured' if settings.is_azure_configured else 'Not configured'}")
print(f"   Observability: {'Azure Monitor' if settings.is_observability_configured else 'Console export'}")
print(f"   OpenAI Deployment: {'Azure OpenAI Deployment Name' if settings.azure_openai_deployment else 'Deployment Name not set'}")

✅ Telemetry initialized
   Azure OpenAI: Configured
   Observability: Azure Monitor
   OpenAI Deployment: Azure OpenAI Deployment Name


## Part 2: Understanding MCP Tools

Before we connect tools to an agent, let's explore the tools available in our workshop.

In [ ]:
# Import the workshop tools
from src.tools import search_web, get_weather, calculate, read_file, write_file

# These tools are also available through the MCP server
from src.tools.mcp_server import TOOLS

print("Available MCP Tools:")
print("="*50)
for tool in TOOLS:
    print(f"\n📦 {tool.name}")
    print(f"   {tool.description}")

In [ ]:
# Let's test a tool directly
print("Testing search_web tool directly:")
print("="*50)

result = await search_web("python programming", max_results=3)

print(f"Found {result['total_found']} results:")
for item in result['results']:
    print(f"\n  📄 {item['title']}")
    print(f"     {item['url']}")
    print(f"     {item['snippet'][:80]}...")

In [ ]:
# Test the calculator tool
print("Testing calculate tool:")
print("="*50)

operations = [
    ("add", 11, 27),
    ("multiply", 7, 8),
    ("sqrt", 144, None),
    ("power", 2, 10),
]

for op, a, b in operations:
    result = await calculate(op, a, b)
    print(f"  {result['expression']} = {result['result']}")

## Part 3: Creating a Simple Agent

Now let's create an agent that can use these tools to accomplish tasks.

In [ ]:
from src.agents import ResearchAgent
from src.tools import search_web, get_weather, calculate

# Create a research agent
agent = ResearchAgent(
    name="workshop_researcher",
    #temperature=0.7,
    max_tool_iterations=5,
)

# Register tool handlers
agent.set_tool_handlers({
    "search_web": search_web,
    "get_weather": get_weather,
    "calculate": calculate,
})

print(f"✅ Created agent: {agent.name}")
print(f"   Model: {agent.model}")
print(f"   Tools registered: {len(agent._tool_handlers)}")

In [ ]:
# Run a simple research task
print("Running research task...")
print("="*50)

task = "What is the weather like in Seattle? Also calculate 15% of 250."

try:
    result = await agent.run(task)
    print("\nAgent Response:")
    print("-"*50)
    print(result)
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\nNote: If Azure OpenAI is not configured, the agent cannot run.")
    print("Check your .env file and ensure AZURE_OPENAI_ENDPOINT is set.")

## Part 4: Observing Agent Behavior with Traces

One of the key aspects of building production agents is observability. Let's examine the traces generated by our agent.

In [ ]:
from src.common.telemetry import get_current_span, get_tracer

tracer = get_tracer("notebook_01")

# Create a traced operation
with tracer.start_as_current_span("demo_trace") as span:
    span.set_attribute("demo.operation", "trace_demo")
    span.set_attribute("demo.notebook", "01_simple_agent_mcp")
    
    # Perform a tool operation within the span
    result = await search_web("azure openai", max_results=2)
    
    span.set_attribute("demo.results_count", result['total_found'])
    span.add_event("search_completed", {"query": "azure openai"})

print("✅ Trace created and exported")
print("")
print("If using Azure Monitor, traces will appear in:")
print("  Azure Portal > Application Insights > Transaction search")
print("")
print("If using console export, check the terminal output for trace data.")

In [ ]:
# Demonstrate trace attributes for agent operations
from src.common.telemetry import create_span_attributes

# These are the attributes we track for agent operations
agent_attrs = create_span_attributes(
    agent_name="demo_agent",
    model="gpt-4o",
    prompt_tokens=150,
    completion_tokens=75,
)

print("Agent Span Attributes:")
print("-"*40)
for key, value in agent_attrs.items():
    print(f"  {key}: {value}")

# Tool attributes
tool_attrs = create_span_attributes(
    tool_name="search_web",
    query="python async",
)

print("\nTool Span Attributes:")
print("-"*40)
for key, value in tool_attrs.items():
    print(f"  {key}: {value}")

## Part 5: Handling MCP Server Unavailability (Edge Case)

In production, external services can fail. Let's explore graceful degradation patterns.

In [ ]:
from src.common.exceptions import MCPError, ToolError

async def resilient_tool_call(tool_name: str, tool_func, **kwargs):
    """Execute a tool with graceful error handling."""
    try:
        result = await tool_func(**kwargs)
        return {"success": True, "data": result}
    except ToolError as e:
        print(f"⚠️ Tool error: {e}")
        return {
            "success": False,
            "error": str(e),
            "fallback": "Using cached or default data"
        }
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return {
            "success": False,
            "error": str(e),
            "fallback": "Service unavailable"
        }

# Test with valid call
print("Testing resilient tool call:")
print("="*50)

result = await resilient_tool_call("search_web", search_web, query="test")
print(f"Success: {result['success']}")

# Test with invalid parameters (will trigger error handling)
result = await resilient_tool_call("calculate", calculate, operation="divide", a=10, b=0)
print(f"\nDivision by zero handled gracefully: {result['success']}")

## Part 6: Hands-On Exercise 🎯

Now it's your turn! Add a new tool to the agent.

### Exercise: Create a Text Analysis Tool

Create a tool that analyzes text and returns:
- Word count
- Character count
- Average word length

In [ ]:
# YOUR CODE HERE: Implement the analyze_text tool

async def analyze_text(text: str) -> dict:
    """Analyze text and return statistics.
    
    Args:
        text: The text to analyze.
        
    Returns:
        Dictionary with word_count, char_count, avg_word_length.
    """
    # TODO: Implement this function
    # Hints:
    # 1. Split text into words using text.split()
    # 2. Count characters with len(text)
    # 3. Calculate average word length
    
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    avg_word_length = sum(len(w) for w in words) / word_count if word_count > 0 else 0
    
    return {
        "word_count": word_count,
        "char_count": char_count,
        "avg_word_length": round(avg_word_length, 2),
    }

# Test your implementation
test_text = "The quick brown fox jumps over the lazy dog"
result = await analyze_text(test_text)
print(f"Analysis of: '{test_text}'")
print(f"  Word count: {result['word_count']}")
print(f"  Character count: {result['char_count']}")
print(f"  Average word length: {result['avg_word_length']}")

In [ ]:
# BONUS: Add your new tool to the agent

# Create a new agent with the additional tool
enhanced_agent = ResearchAgent(
    name="enhanced_researcher",
    system_prompt="""You are a research assistant with text analysis capabilities.
    
    You can:
    - Search the web for information
    - Perform calculations
    - Analyze text for statistics
    
    When asked about text, use the analyze_text tool.""",
)

enhanced_agent.set_tool_handlers({
    "search_web": search_web,
    "calculate": calculate,
    "analyze_text": analyze_text,  # Your new tool!
})

print(f"✅ Enhanced agent created with {len(enhanced_agent._tool_handlers)} tools")

## Summary

In this scenario, you learned:

1. **MCP Tools**: How to create and use tools that follow the Model Context Protocol
2. **Agent Creation**: How to build agents with Azure OpenAI that can use tools
3. **Observability**: How to add OpenTelemetry tracing to observe agent behavior
4. **Error Handling**: How to handle tool failures gracefully
5. **Tool Extension**: How to add new tools to extend agent capabilities

## Key Concepts

- **MCP** provides a standardized way to expose tools to AI models
- **Telemetry** helps you understand what your agent is doing
- **Graceful degradation** ensures your agent remains useful even when tools fail

## Next Steps

Continue to **02_agui_interface.ipynb** to learn about building user interfaces for agents using the AG-UI protocol.

In [ ]:
# Cleanup
await agent.close()
print("✅ Scenario 1 complete!")